In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scenedataset.scenedataset import SceneDataset

/home/abdalla/scenedataset/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = SceneDataset(paths = ["/home/abdalla/TorchGANime/data/kny/raw_videos/01.mkv"], show_progress=True)

FileNotFoundError: [Errno 2] No such file or directory: 'ffprobe'